In [1]:
def smp_probe(filename: str) -> bool:
    with open(filename, "rb") as f:
        return f.read(9) == b"file=samp"


def smp_headers(filename: str):
    with open(filename, "rb") as f:
        f.seek(0)
        raw_headers = f.read(1024)
        raw_headers = raw_headers.rstrip(b'\x00')
        asc_headers = raw_headers.decode("ascii")
        asc_headers.rstrip('\x00')
        tmp = [a for a in asc_headers.split("\r\n")]
        back = -1
        while abs(back) > len(tmp) + 1:
            if tmp[back] == '=':
                break
            back -= 1
        tmp = tmp[0:back-1]
        return dict(a.split("=") for a in tmp)

In [2]:
example = "/Users/joregan/Playing/waxholm/scenes_formatted/fp2060/fp2060.11.04.smp"

In [5]:
with open(example, "rb") as egf:
    egf.seek(0)
    blah = egf.read(1024)
    a = egf.tell()
a

1024

In [ ]:
def input_args(
        save_dir,
        tensorboard_logdir,
        data,
        empty_cache_freq=0,
        freeze_updates=2000,  # Freeze all but the output layer
        max_tokens_per_minibatch=6400000,
        dropout=0.1,
        random_seed=5728395,
        weight_decay=0.0
):
    # This is pretty clunky, but still less clunky than bash imo
    n_gpu = n_gpu or torch.cuda.device_count() or 1
    update_freq = update_freq or math.ceil(max_tokens_per_minibatch / max_tokens / n_gpu)
    args_string = f"""
      {data}
      --save-dir {save_dir}
      --tensorboard-logdir {tensorboard_logdir}
      --empty-cache-freq {empty_cache_freq}
      --max-tokens-valid {max_tokens}
      --feature-grad-mult 0.0
      --freeze-finetune-updates {freeze_updates}
      --dropout {dropout}
      --seed {random_seed}
    """

In [6]:
%%writefile config.yaml
common:
  fp16: true
  log_format: json
  log_interval: 50

checkpoint:
  no_epoch_checkpoints: true
  best_checkpoint_metric: uer

task:
  _name: audio_finetuning
  data: ???
  max_sample_size: 1120000
  normalize: false
  labels: ltr

dataset:
  num_workers: 1
  max_tokens: 1120000
  skip_invalid_size_inputs_valid_test: true
  validate_after_updates: 1000
  validate_interval: 1
  valid_subset: valid

distributed_training:
  ddp_backend: no_c10d
  distributed_world_size: 1

criterion:
  _name: ctc
  zero_infinity: true

optimization:
  max_update: 12000
  lr: [0.00005]
  sentence_avg: true
  weight_decay: 0.0
  update_freq: []

optimizer:
  _name: adam
  adam_betas: (0.9,0.98)
  adam_eps: 1e-08

lr_scheduler:
  _name: tri_stage
  phase_ratio: [0.33, 0.33, 0.33]
  final_lr_scale: 0.05

model:
  _name: wav2vec_ctc
  w2v_path: ???
  apply_mask: true
  mask_prob: 0.65
  mask_channel_prob: 0.25
  mask_channel_length: 64
  layerdrop: 0.1
  activation_dropout: 0.1
  feature_grad_mult: 0.0
  freeze_finetune_updates: 0
  final_dropout: 0.1
  attention_dropout: 0.1

Writing config.yaml


In [8]:
%%writefile new.yaml
# @package _group_

common:
  fp16: true
  log_format: json
  log_interval: 200

checkpoint:
  save_interval: 1000
  save_interval_updates: 50
  keep_interval_updates: 1
  no_epoch_checkpoints: true
  best_checkpoint_metric: uer

task:
  _name: audio_pretraining
  data: ???
  normalize: false
  labels: ltr

dataset:
  num_workers: 1
  max_tokens: 1120000
  skip_invalid_size_inputs_valid_test: true
  validate_after_updates: 1000
  validate_interval: 1
  valid_subset: valid

distributed_training:
  ddp_backend: no_c10d
  distributed_world_size: 1

criterion:
  _name: ctc
  zero_infinity: true

optimization:
  max_update: 12000
  lr: [0.00005]
  sentence_avg: true
  update_freq: [4]

optimizer:
  _name: adam
  adam_betas: (0.9,0.98)
  adam_eps: 1e-08

lr_scheduler:
  _name: tri_stage
  phase_ratio: [0.1, 0.4, 0.5]
  final_lr_scale: 0.05

model:
  _name: wav2vec_ctc
  w2v_path: ???
  apply_mask: true
  mask_prob: 0.65
  mask_channel_prob: 0.25
  mask_channel_length: 64
  layerdrop: 0.1
  activation_dropout: 0.1
  feature_grad_mult: 0.0
  final_dropout: 0.1
  attention_dropout: 0.1
  freeze_finetune_updates: 0


Overwriting new.yaml
